# QA on a single research article

> Building a simple chatbot to allow the user to ask questions about a single paper. 

In [1]:
#| default_exp tools.single_paper_qa

In [2]:
#| export

import os
import pandas as pd

import alhazen.utils.jats_text_extractor as te 
import re
import os

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
from llama_index import Document, ServiceContext, set_global_service_context
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.node_parser import SentenceWindowNodeParser
import requests

from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
)

import gradio as gr
from databricks import sql
import os

import fire

In [3]:
#| export 

GGUF_LOOKUP_URL = {
    "llama-2-70b-chat.Q5_K_M.gguf": "https://huggingface.co/TheBloke/Llama-2-70B-chat-GGUF/resolve/main/llama-2-70b-chat.Q4_K_M.gguf",
    "llama-2-13b-chat.Q5_K_M.gguf": "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q5_K_M.gguf",
    "llama-2-7b-chat.Q5_K_M.gguf": "https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF/resolve/main/llama-2-7b-chat.Q5_K_M.gguf"
}

class SingleOpenAccessArticleChatBot:

    def __init__(self, temp_dir='/tmp/alhazen',
                embed_model_name="BAAI/bge-small-en-v1.5", 
                gguf_file_name='llama-2-13b-chat.Q5_K_M.gguf',
                temperature=0.1,
                max_new_tokens=256,
                context_window=3900,
                sentence_window_size=3):

        self.embed_model = HuggingFaceEmbedding(model_name=embed_model_name)
        
        if temp_dir[-1:] != '/':
            temp_dir += '/'
        self.temp_dir = temp_dir

        # download relevant file to local disk if not already there
        if os.path.exists(temp_dir+gguf_file_name) is False:
            print('Downloading GGUF file: ' + gguf_file_name)
            os.makedirs(temp_dir, exist_ok=True)

            # download file from HuggingFace but show progress bar  
            r = requests.get(GGUF_LOOKUP_URL[gguf_file_name], stream=True)
            with open(temp_dir+gguf_file_name, 'wb') as f:
                total_length = int(r.headers.get('content-length'))
                dl = 0
                for chunk in r.iter_content(chunk_size=1024*1024):
                    dl += len(chunk)
                    f.write(chunk)
                    done = int(50 * dl / total_length)
                    print('\r[{}{}]'.format('=' * done, ' ' * (50-done)), end='')
                print('\n')
            print('Download complete.')
                        
        self.llm = LlamaCPP( 
            model_path=temp_dir+gguf_file_name,
            temperature=temperature,
            max_new_tokens=max_new_tokens,
            context_window=context_window,
            generate_kwargs={},
            model_kwargs={"n_gpu_layers": 1},
            messages_to_prompt=messages_to_prompt,
            completion_to_prompt=completion_to_prompt,
            verbose=True)     

        # create the sentence window node parser w/ default settings
        self.node_parser = SentenceWindowNodeParser.from_defaults(
            window_size=sentence_window_size,
            window_metadata_key="window",
            original_text_metadata_key="original_text",
            )
        self.embed_model = HuggingFaceEmbedding(model_name=embed_model_name)

        # create a service context
        self.service_context = ServiceContext.from_defaults(
            llm=self.llm,
            embed_model=self.embed_model,
            node_parser=self.node_parser)

    def get_crossref_data_from_doi(self, doi):

        stem = 'https://api.crossref.org/v1/works/'
        url = stem + doi
        json_data = requests.get(url).text
        return json_data

    def get_ft_url_from_doi(self, doi):
        
        try:
            '''<<< NOTES ABOUT HOW TO CONNECT TO DATABRICKS USING SQLALCHEMY >>>
            engine = create_engine(
                'databricks://token:'+os.environ['DB_TOKEN']+ 
                '@czi-shared-infra-czi-sci-general-prod-databricks.cloud.databricks.com'+
                '?http_path=/sql/1.0/warehouses/1c4df94f2f1a6305')
            sql = 'SELECT * FROM BLAHBLAHBLAH
            with engine.connect() as con:
                stmt = db.text(get_ft_url_from_doi_sql)
                rs = con.execute(stmt)
                df = pd.DataFrame(rs.fetchall(), columns=rs.keys())'''

            get_ft_url_from_doi_sql = '''
                SELECT DISTINCT p.pmc_id, p.doi, YEAR(p.publication_date) as year, p.title, p.abstract, p.full_text_format, p.full_text_url, a.last_name
                FROM scipubstore.ingestion.papers as p 
                    JOIN scipubstore.ingestion.authors as a on (p.paper_id=a.paper_id) 
                WHERE p.doi = '{}' and a.author_index=1
                ORDER BY p.full_text_url DESC
            '''.format(doi)

            df = pd.DataFrame()
            with sql.connect(server_hostname = 'czi-shared-infra-czi-sci-general-prod-databricks.cloud.databricks.com',
                                http_path = '/sql/1.0/warehouses/1c4df94f2f1a6305',
                                access_token = os.getenv("DB_TOKEN")) as connection:

                with connection.cursor() as cursor:
                    cursor.execute(get_ft_url_from_doi_sql)
                    result = cursor.fetchall()
                    df = pd.DataFrame([row.asDict() for row in result])

        except:

            msg = 'Error attempting to query Databricks for URL data, did you set the DB_TOKEN environment variable?'
            raise Exception(msg)

        return df
    
    def run_gradio(self):

        with gr.Blocks() as demo:
            gr.Markdown('# Alhazen - Single Paper Q/A Agent v.0.1')

            def load_jats_paper(doi):
                df = self.get_ft_url_from_doi(doi)
                print(df.columns)
                if df.shape[0] == 0:
                    return('No paper found with that DOI')
                title = df['title'].values[0]
                first_author = df['last_name'].values[0]
                
                year = df['year'].values[0]  
                url = df['full_text_url'].values[0]
                xml = requests.get(url).text
                doc = te.NxmlDoc(doi, xml)
                df1 = doc.build_simple_document_dataframe()
                df_text = df1[df1['FIG_REF'] == '']
                for i, row in df_text.iterrows():
                    if row.TAG != 'p' and row.PLAIN_TEXT[-1:]!='.':
                        row.PLAIN_TEXT += '.'
                text_list = df_text['PLAIN_TEXT'].values.tolist()
                text_length = sum([len(t) for t in text_list])
                documents = [Document(text=t) for t in text_list]
                self.index = VectorStoreIndex.from_documents(documents, service_context=self.service_context)
                self.query_engine = self.index.as_query_engine()
                return('Loaded paper: ' + first_author + ' (' + str(year) + ') "' + title + '\n' + str(text_length))

            def chat(message, chat_history):
                bot_message = self.query_engine.query(message)
                chat_history.append((message, bot_message.response))
                return "", chat_history

            with gr.Tab("Paper"):
                doi_tb = gr.Textbox(label="Load DOI...",value='10.1101/2022.01.23.477440')
                index_status = gr.Textbox(label='Index Status: ')
                text_button = gr.Button("Download File  Document")
                text_button.click(load_jats_paper, doi_tb, index_status)

            with gr.Tab("Knowledge Bot"):
                chatbot = gr.Chatbot()
                msg = gr.Textbox()
                clear = gr.ClearButton([msg, chatbot])
                msg.submit(chat, [msg, chatbot], [msg, chatbot])

        demo.queue().launch(debug = True) 

In [2]:
#| export

def chatbot(temp_dir='/tmp/alhazen',
                embed_model_name="BAAI/bge-small-en-v1.5", 
                gguf_file_name='llama-2-13b-chat.Q5_K_M.gguf',
                temperature=0.1,
                max_new_tokens=256,
                context_window=3900,
                sentence_window_size=3):
    
    chatbot = SingleOpenAccessArticleChatBot(temp_dir=temp_dir,
                embed_model_name=embed_model_name, 
                gguf_file_name=gguf_file_name,
                temperature=temperature,
                max_new_tokens=max_new_tokens,
                context_window=context_window,
                sentence_window_size=sentence_window_size)
    chatbot.run_gradio()

## How to create a chatbot. 

1. Download the GGUF file you want the chatbot to use for Llama2 and put it in a temp directory
2. Set the environment variable `DB_TOKEN` to your databricks access token 
   1. You need access to the Databricks SQL warehouse named `sci_general_sql` (ID: 1c4df94f2f1a6305) on the general czi-prod databricks instance: `https://czi-shared-infra-czi-sci-general-prod-databricks.cloud.databricks.com/` 
3. Execute the following code: 

```python
from alhazen.gradio import SingleOpenAccessArticleChatBot

chatbot = SingleOpenAccessArticleChatBot()
chatbot.run_gradio()
```
4. Navigate to the URL that is printed out.

This should load a page with instructions that you can follow.